|평가문항|상세기준|
|:------|:---|
|1. 모델과 데이터를 정상적으로 불러오고, 작동하는 것을 확인하였다. |klue/bert-base를 NSMC 데이터셋으로 fine-tuning하여, 모델이 정상적으로 작동하는 것을 확인하였다.|
|2. Preprocessing을 개선하고, fine-tuning을 통해 모델의 성능을 개선시켰다. | Validation accuracy를 90%이상으로 개선하였다.|
|3. 모델학습에 Bucketing을 성공적으로 적용하고, 그 결과를 비교분석하였다. | Bucketing Task를 수행하여 fine-tuning 시 연산 속도와 모델 성능가느이 trade-off 관계가 발생하는지 여부를 확인하고, 분서갛ㄴ 결과를 제시하였다.|

### Step 0. 패키지 호출

In [1]:
# !pip install transformers==4.4.2

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import transformers
import datasets
from datasets import load_dataset, DatasetDict, Dataset
import tensorflow_datasets as tfds

from transformers import AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

In [3]:
!nvidia-smi

Fri Sep 15 04:00:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Step 1. NSMC 데이터 분석 및 Huggingface dataset 구성

데이터셋은 깃허브에서 다운받거나, Huggingface datasets에서 가져올 수 있습니다. 앞에서 배운 방법들을 활용해봅시다!

In [4]:
huggingface_dataset = load_dataset('nsmc')
print(huggingface_dataset)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [5]:
train = huggingface_dataset['train']
cols = train.column_names
cols

['id', 'document', 'label']

In [6]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




### Step 2. klue/bert-base model 및 tokenizer 불러오기

In [7]:
class DataSet():
    def __init__(self, dataset_name, model_name, padding='max_length'):
        super(DataSet, self).__init__()
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        
        self.padding = padding
        dataset = self._set(dataset_name)
        
        self.train = dataset['train']
        self.test = dataset['test']
        self.valid = dataset['valid']
                                        
            
    def transform(self, data):
        return self.tokenizer(
            data['document'],
            truncation=True,
            padding=self.padding,
#             padding='max_length',
            return_token_type_ids=False,
        )
       
        
    def _set(self, dataset_name):
        data = load_dataset(dataset_name)
        train_valid = data['train'].train_test_split(test_size=0.2)
                
        return DatasetDict({
            'train': train_valid['train'],
            'valid': train_valid['test'],
            'test': data['test']
        }).map(self.transform, batched=True)

In [8]:
dataset_name = 'nsmc'
model_name = 'klue/bert-base'
output_dir = './results'
    
dataset = DataSet(dataset_name, model_name, padding=False)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached split indices for dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-a3c98fba040dae08.arrow and /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-4552e18e170dc7b2.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-9957dc54dc1c0b75.arrow


  0%|          | 0/30 [00:00<?, ?ba/s]

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-cb59f68dd3aff018.arrow


### Step 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해보기

In [9]:
class Classifier():
    def __init__(self, model_name, dataset, training_arguments):
        super(Classifier, self).__init__()
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
        
        self.dataset = dataset
        self.trainer = self._set(training_arguments)
    
    
    def compute_metrics(self, pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        return { 'accuracy': accuracy_score(labels, preds) }
    
    
    def _set(self, training_arguments):
        return Trainer(
            model=self.model,           
            args=training_arguments,           
            train_dataset=self.dataset.train,
            eval_dataset=self.dataset.valid,       
            compute_metrics=self.compute_metrics,
            data_collator=DataCollatorWithPadding(tokenizer=self.tokenizer)
        )
    
    
    def fine_tuning(self):
        return self.trainer.train()
        
    
    def evaluate(self):
        return self.trainer.evaluate(self.dataset.test)

#### model1: batch_size 16

In [10]:
model1 = Classifier(
    model_name,
    dataset,
    TrainingArguments(
        output_dir,
        evaluation_strategy = 'epoch',
        learning_rate = 2e-5,
        per_device_train_batch_size = 16,
        per_device_eval_batch_size = 16,
        num_train_epochs = 1,
        warmup_steps = 1000,
        weight_decay = 0.01,
        fp16 = True,
    ))

model1.fine_tuning()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.284400,0.291624,0.903533,107.014100,280.337000


/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1081: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(
/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1081: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(
/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1081: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that 

TrainOutput(global_step=15000, training_loss=0.3217869557698568, metrics={'train_runtime': 2116.6689, 'train_samples_per_second': 7.087, 'total_flos': 4100680893586464.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 1699512, 'init_mem_gpu_alloc_delta': 443266560, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1092153, 'train_mem_gpu_alloc_delta': 1332549120, 'train_mem_cpu_peaked_delta': 103720035, 'train_mem_gpu_peaked_delta': 1037661696})

In [11]:
model1.evaluate()

{'eval_loss': 0.29693958163261414,
 'eval_accuracy': 0.9013,
 'eval_runtime': 177.0711,
 'eval_samples_per_second': 282.372,
 'epoch': 1.0,
 'eval_mem_cpu_alloc_delta': 859349,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 3056413,
 'eval_mem_gpu_peaked_delta': 34402304}

### Step 4. Fine-tuning을 통하여 모델 성능(accuracy)향상시키기

- 데이터 전처리, TrainingArguments 등을 조정하여 모델의 정확도를 90% 이상으로 끌어올려봅시다.

#### model2: accumulation_steps 적용

In [18]:
model2 = Classifier(
    model_name,
    dataset,
    TrainingArguments(
        output_dir, 
        evaluation_strategy="epoch",
        learning_rate=2e-5,   
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        gradient_accumulation_steps=2,
        eval_accumulation_steps=2,
        num_train_epochs=1,
        warmup_steps=1000, 
        weight_decay=0.01,                 
        fp16=True,
    ))

model2.fine_tuning()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.248700,0.249789,0.901733,110.500000,271.493000


/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1081: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(
/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1081: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(
/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1081: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that 

TrainOutput(global_step=7500, training_loss=0.29384472147623697, metrics={'train_runtime': 1549.6869, 'train_samples_per_second': 4.84, 'total_flos': 4100680893586464.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 1793072, 'init_mem_gpu_alloc_delta': 443266560, 'init_mem_cpu_peaked_delta': 378242, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 505311, 'train_mem_gpu_alloc_delta': 1329796608, 'train_mem_cpu_peaked_delta': 103761879, 'train_mem_gpu_peaked_delta': 1058182144})

In [20]:
model2.evaluate()

{'eval_loss': 0.25234317779541016,
 'eval_accuracy': 0.9013,
 'eval_runtime': 183.344,
 'eval_samples_per_second': 272.711,
 'epoch': 1.0,
 'eval_mem_cpu_alloc_delta': 877795,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 3070476,
 'eval_mem_gpu_peaked_delta': 34960384}

### Step 5. Bucketing을 작용하여 학습시키고, Step 4의 결과와의 비교

아래 링크를 바탕으로 bucketing과 dynamic padding이 무엇인지 알아보고, 이들을 적용하여 model을 학습시킵니다.

- Data Collator
- Trainer.TrainingArguments 의 group_by_length

STEP 4에 학습한 결과와 bucketing을 적용하여 학습시킨 결과를 비교해보고, 모델 성능 향상과 훈련 시간 두 가지 측면에서 각각 어떤 이점이 있는지 비교해봅시다.

#### model3: group_by_length 적용

In [21]:
model3 = Classifier(
    model_name,
    dataset,
    TrainingArguments(
        output_dir, 
        evaluation_strategy="epoch",
        learning_rate=2e-5,   
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
#         gradient_accumulation_steps=2,
#         eval_accumulation_steps=2,
        num_train_epochs=1,
        warmup_steps=1000, 
        weight_decay=0.01,                 
        fp16=True,
#         group_by_length=True
    ))

model3.fine_tuning()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.286300,0.298082,0.902367,106.828800,280.823000


/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1081: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(
/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1081: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(
/opt/conda/lib/python3.9/site-packages/transformers/trainer.py:1081: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that 

TrainOutput(global_step=15000, training_loss=0.32392432454427084, metrics={'train_runtime': 2118.8026, 'train_samples_per_second': 7.079, 'total_flos': 4100680893586464.0, 'epoch': 1.0, 'init_mem_cpu_alloc_delta': 46909, 'init_mem_gpu_alloc_delta': 443266560, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 512960, 'train_mem_gpu_alloc_delta': 1329796608, 'train_mem_cpu_peaked_delta': 103765442, 'train_mem_gpu_peaked_delta': 1057494016})

In [23]:
model3.evaluate()

{'eval_loss': 0.2991721034049988,
 'eval_accuracy': 0.90218,
 'eval_runtime': 354.8857,
 'eval_samples_per_second': 140.89,
 'epoch': 1.0,
 'eval_mem_cpu_alloc_delta': 859196,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 3054902,
 'eval_mem_gpu_peaked_delta': 34402304}

### 회고록

- 우연히 첫번쨰 모델에서 정확도가 90%가 넘었다. 
- 따라서 두번째 모델에서 파라미터 튜닝을 시도해봐도 첫번쨰 모델과 크게 달라진건 없었다.
- Bucketing task를 수행했을때와 fine-tuning 시 연산 속도와 모델성능간의 trade-off관계가 발생하긴 한다. 시간이 훨씬 많이 걸린다. 하지만 전반적이 성능 비슷하다.